# 🧭 Lab 3: Planning Patterns
## Module 2 - ReAct, Plan-Execute, and Tree-of-Thought

**Duration:** 30 minutes

**Objectives:**
- Implement the ReAct (Reasoning + Acting) pattern
- Implement the Plan-Execute pattern
- Compare cost and effectiveness

**Banking Scenario:** Fraud investigation (ReAct) vs Dispute resolution (Plan-Execute)

---

In [ ]:
!pip install openai -q

In [ ]:
import os
import json
from typing import List, Dict

# =============================================================================
# GOOGLE COLAB SETUP - Add these secrets (click 🔑 icon):
#   - AZURE_OPENAI_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_DEPLOYMENT
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing")
except Exception:
    print("⚠️ Running in DEMO MODE")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Client ready")

## Part 1: ReAct Pattern - Fraud Investigation

**ReAct** = **Re**asoning + **Act**ing

The agent thinks, acts, observes, and repeats until it has enough information.

In [ ]:
investigation_tools = [
    {"type": "function", "function": {"name": "get_transaction_history", "description": "Get customer transaction history", "parameters": {"type": "object", "properties": {"customer_id": {"type": "string"}, "days": {"type": "integer"}}, "required": ["customer_id"]}}},
    {"type": "function", "function": {"name": "get_account_changes", "description": "Get recent account changes", "parameters": {"type": "object", "properties": {"customer_id": {"type": "string"}}, "required": ["customer_id"]}}},
    {"type": "function", "function": {"name": "get_login_history", "description": "Get login attempts and devices", "parameters": {"type": "object", "properties": {"customer_id": {"type": "string"}}, "required": ["customer_id"]}}},
    {"type": "function", "function": {"name": "create_fraud_recommendation", "description": "Create fraud recommendation", "parameters": {"type": "object", "properties": {"customer_id": {"type": "string"}, "risk_level": {"type": "string"}, "recommendation": {"type": "string"}, "reasoning": {"type": "string"}}, "required": ["customer_id", "risk_level", "recommendation", "reasoning"]}}}
]

def execute_investigation_tool(name: str, args: dict) -> dict:
    responses = {
        "get_transaction_history": {"avg_transaction": 2500, "max_transaction": 8000, "international_transfers": 0, "flagged_transaction": {"amount": 45000, "destination": "Nigeria"}},
        "get_account_changes": {"email_changed": "3 days ago", "phone_changed": "3 days ago", "password_reset": "2 days ago"},
        "get_login_history": {"usual_location": "San Francisco", "recent_logins": [{"location": "San Francisco", "days_ago": 5}, {"location": "Lagos, Nigeria", "days_ago": 0}]},
        "create_fraud_recommendation": {"case_id": "FRAUD-2024-001", "status": "created"}
    }
    return responses.get(name, {"error": "Unknown tool"})

In [ ]:
def run_demo_react(alert: str) -> dict:
    """Demo mode for ReAct pattern"""
    print("\n⚠️ DEMO MODE - Simulating ReAct pattern\n")
    trace = []
    
    print("--- Iteration 1 ---")
    print("💭 Thought: I need to check the customer's transaction history first")
    print("⚡ Action: get_transaction_history")
    result = execute_investigation_tool("get_transaction_history", {})
    print(f"👁️ Observation: {result}")
    trace.append({"tool": "get_transaction_history", "result": result})
    
    print("\n--- Iteration 2 ---")
    print("💭 Thought: The transaction is 5.6x higher than max. Check account changes.")
    print("⚡ Action: get_account_changes")
    result = execute_investigation_tool("get_account_changes", {})
    print(f"👁️ Observation: {result}")
    trace.append({"tool": "get_account_changes", "result": result})
    
    print("\n--- Iteration 3 ---")
    print("💭 Thought: Multiple account changes recently. Check login history.")
    print("⚡ Action: get_login_history")
    result = execute_investigation_tool("get_login_history", {})
    print(f"👁️ Observation: {result}")
    trace.append({"tool": "get_login_history", "result": result})
    
    print("\n✅ Investigation complete")
    
    return {
        "recommendation": """**RECOMMENDATION: BLOCK**\n\nRisk Level: CRITICAL\n\nFindings:\n1. Transaction 5.6x higher than historical max\n2. Email, phone, password all changed in last 3 days\n3. Login from Nigeria (unusual location)\n\nAction: Block transaction, contact customer immediately.""",
        "trace": trace,
        "iterations": 4,
        "total_tokens": 1500
    }

In [ ]:
def react_agent(alert: str, max_iterations: int = 6) -> dict:
    print("\n" + "="*60)
    print("🔄 ReAct Agent: Fraud Investigation")
    print("="*60)
    
    if DEMO_MODE or client is None:
        return run_demo_react(alert)
    
    messages = [
        {"role": "system", "content": """You are a fraud investigation agent using ReAct pattern.\nFor each step: THOUGHT → ACTION → OBSERVATION\nContinue until you have enough evidence."""},
        {"role": "user", "content": f"Investigate: {alert}"}
    ]
    
    trace = []
    total_tokens = 0
    
    try:
        for i in range(max_iterations):
            print(f"\n--- Iteration {i+1} ---")
            response = client.chat.completions.create(model=MODEL_NAME, messages=messages, tools=investigation_tools)
            total_tokens += response.usage.total_tokens
            msg = response.choices[0].message
            messages.append(msg)
            
            if msg.content:
                print(f"💭 Thought: {msg.content[:150]}..." if len(msg.content) > 150 else f"💭 Thought: {msg.content}")
            
            if msg.tool_calls:
                for tc in msg.tool_calls:
                    tool_name = tc.function.name
                    tool_args = json.loads(tc.function.arguments)
                    print(f"⚡ Action: {tool_name}")
                    result = execute_investigation_tool(tool_name, tool_args)
                    print(f"👁️ Observation: {json.dumps(result)[:100]}...")
                    trace.append({"tool": tool_name, "result": result})
                    messages.append({"role": "tool", "tool_call_id": tc.id, "content": json.dumps(result)})
            else:
                print(f"\n✅ Investigation complete after {i+1} iterations")
                return {"recommendation": msg.content, "trace": trace, "iterations": i + 1, "total_tokens": total_tokens}
        
        return {"recommendation": "Max iterations", "trace": trace, "iterations": max_iterations, "total_tokens": total_tokens}
    except Exception as e:
        print(f"\n⚠️ API Error: {e}")
        return run_demo_react(alert)

In [ ]:
alert = """FRAUD ALERT - HIGH PRIORITY
Customer ID: C-789
Transaction: Wire transfer $45,000 to Nigeria
Flagged by: Amount threshold + International destination
Customer tier: Standard (typical < $10K)"""

react_result = react_agent(alert)

print("\n" + "="*60)
print("FINAL RECOMMENDATION:")
print("="*60)
print(react_result["recommendation"])
print(f"\n📊 Stats: {react_result['iterations']} iterations, {react_result['total_tokens']} tokens")

## Part 2: Plan-Execute Pattern - Dispute Resolution

**Plan-Execute** = Create a plan first, then execute steps in order.

In [ ]:
def execute_dispute_tool(name: str, args: dict) -> dict:
    responses = {
        "verify_customer_identity": {"verified": True, "customer_name": "John Smith"},
        "get_transaction_details": {"amount": 299.99, "merchant": "AMZN MKTP US", "date": "2024-01-15"},
        "check_merchant_history": {"dispute_rate": 0.02, "avg_resolution_days": 5},
        "create_dispute_case": {"case_id": "DSP-12345", "status": "created"}
    }
    return responses.get(name, {"error": "Unknown"})

In [ ]:
def run_demo_plan_execute(task: str) -> dict:
    """Demo mode for Plan-Execute pattern"""
    print("\n⚠️ DEMO MODE - Simulating Plan-Execute\n")
    
    plan = {"steps": [
        {"step": 1, "tool": "verify_customer_identity", "purpose": "Confirm customer"},
        {"step": 2, "tool": "get_transaction_details", "purpose": "Get transaction info"},
        {"step": 3, "tool": "check_merchant_history", "purpose": "Check merchant"},
        {"step": 4, "tool": "create_dispute_case", "purpose": "Create case"}
    ]}
    
    print("📋 Plan created:")
    for s in plan['steps']:
        print(f"   {s['step']}. {s['tool']} - {s['purpose']}")
    
    print("\n--- Executing Plan ---")
    results = []
    for step in plan['steps']:
        result = execute_dispute_tool(step['tool'], {})
        results.append({"step": step['step'], "result": result})
        print(f"   ✅ Step {step['step']}: {result}")
    
    return {
        "plan": plan,
        "results": results,
        "summary": "Dispute case DSP-12345 created for $299.99 charge at AMZN MKTP US. Customer verified. Expected resolution: 5-7 days.",
        "llm_calls": 2,
        "total_tokens": 800
    }

In [ ]:
def plan_execute_agent(task: str) -> dict:
    print("\n" + "="*60)
    print("📋 Plan-Execute Agent: Dispute Resolution")
    print("="*60)
    
    if DEMO_MODE or client is None:
        return run_demo_plan_execute(task)
    
    total_tokens = 0
    
    try:
        # Step 1: Create plan
        print("\n--- Creating Plan ---")
        plan_response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "Create a step-by-step plan as JSON: {\"steps\": [{\"step\": 1, \"tool\": \"name\", \"purpose\": \"why\"}]}\nTools: verify_customer_identity, get_transaction_details, check_merchant_history, create_dispute_case"},
                {"role": "user", "content": f"Plan for: {task}"}
            ],
            response_format={"type": "json_object"}
        )
        total_tokens += plan_response.usage.total_tokens
        plan = json.loads(plan_response.choices[0].message.content)
        
        print(f"📋 Plan: {len(plan['steps'])} steps")
        for s in plan['steps']:
            print(f"   {s['step']}. {s['tool']}")
        
        # Step 2: Execute
        print("\n--- Executing ---")
        results = []
        for step in plan['steps']:
            result = execute_dispute_tool(step['tool'], step.get('params', {}))
            results.append({"step": step['step'], "result": result})
            print(f"   ✅ {step['tool']}: {result}")
        
        # Step 3: Summarize
        summary_response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": f"Summarize: {json.dumps(results)}"}]
        )
        total_tokens += summary_response.usage.total_tokens
        
        return {"plan": plan, "results": results, "summary": summary_response.choices[0].message.content, "llm_calls": 2, "total_tokens": total_tokens}
    except Exception as e:
        print(f"\n⚠️ API Error: {e}")
        return run_demo_plan_execute(task)

In [ ]:
task = """Process dispute for customer C-456:
- Transaction ID: TXN-299
- Amount: $299.99 at AMZN MKTP US
- Customer claims: Did not make this purchase"""

plan_result = plan_execute_agent(task)

print("\n" + "="*60)
print("SUMMARY:")
print("="*60)
print(plan_result["summary"])
print(f"\n📊 Stats: {plan_result['llm_calls']} LLM calls, {plan_result['total_tokens']} tokens")

## Part 3: Compare Patterns

In [ ]:
print("\n" + "="*60)
print("PATTERN COMPARISON")
print("="*60)
print(f"""
| Metric          | ReAct              | Plan-Execute       |
|-----------------|--------------------|--------------------|""")
print(f"| Iterations      | {react_result['iterations']}                  | {plan_result['llm_calls']}                  |")
print(f"| Tokens          | {react_result['total_tokens']}              | {plan_result['total_tokens']}               |")
print(f"| Best For        | Exploratory        | Structured         |")

---
## ✅ Lab 3 Complete!

**Key Takeaways:**
- **ReAct**: Best for exploratory tasks where you don't know what you'll find
- **Plan-Execute**: Best for structured tasks with known steps
- ReAct uses more tokens but can discover unexpected patterns

**Next:** Open `04_memory_systems.ipynb`